# Importar librerias

In [114]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from joblib import dump
from energy_consumption_architecture.utils.paths import data_dir
from xgboost import XGBRegressor


# Funciones auxiliares

In [99]:
# Function to calculate the correlation matrix and plot a heatmap
def matriz_correlacion(dataset, target):
    corr_matrix = dataset.corr()
    corr_matrix[target].sort_values(ascending=False)
    cm_red_blue = mpl.colormaps['RdBu']
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    f, ax = plt.subplots(figsize=(12, 10))
    sns.heatmap(corr_matrix, mask=mask, annot=True, fmt=".2f", cmap=cm_red_blue, vmax=1, vmin=-1, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})
    plt.title('Heatmap with Numerical Values and Colors')
    plt.show()

In [100]:
# Calcular el VIF para cada característica
def calculate_vif(df):
    vif_data = pd.DataFrame()
    vif_data["feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif_data

In [101]:
# Remover iterativamente características con VIF alto
def remove_high_vif_features(df, threshold=10):
    while True:
        vif_data = calculate_vif(df)
        max_vif = vif_data['VIF'].max()
        if max_vif > threshold:
            feature_to_remove = vif_data.loc[vif_data['VIF'] == max_vif, 'feature'].values[0]
            df = df.drop(columns=[feature_to_remove])
        else:
            break
    return df, vif_data

In [102]:
# Extraer características temporales
def extract_time_features(df):
    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    return df

In [103]:
# Dividir en entrenamiento y prueba para series de tiempo
def time_series_train_test_split(df, target, test_size=0.2):
    n_test = int(len(df) * test_size)
    train = df[:-n_test]
    test = df[-n_test:]
    X_train, y_train = train.drop(columns=[target]), train[target]
    X_test, y_test = test.drop(columns=[target]), test[target]
    return X_train, X_test, y_train, y_test

In [104]:
# Entrenar y evaluar un modelo
def train_evaluate_model(model, X_train, X_test, y_train, y_test):
    X_train_array, X_test_array = X_train.values, X_test.values  # Convertir a matrices NumPy
    y_train_array, y_test_array = y_train.values, y_test.values
    
    model.fit(X_train_array, y_train_array)
    y_train_pred = model.predict(X_train_array)
    y_test_pred = model.predict(X_test_array)
    
    rmse_train = np.sqrt(mean_squared_error(y_train_array, y_train_pred))
    mae_train = mean_absolute_error(y_train_array, y_train_pred)
    r2_train = r2_score(y_train_array, y_train_pred)
    
    rmse_test = np.sqrt(mean_squared_error(y_test_array, y_test_pred))
    mae_test = mean_absolute_error(y_test_array, y_test_pred)
    r2_test = r2_score(y_test_array, y_test_pred)
    
    return {
        "Model": model,
        "Train RMSE": rmse_train,
        "Train MAE": mae_train,
        "Train R2": r2_train,
        "Test RMSE": rmse_test,
        "Test MAE": mae_test,
        "Test R2": r2_test
    }


In [105]:
# Evaluar todos los modelos para un cluster
def evaluate_models_for_cluster(X_train, X_test, y_train, y_test, models):
    results = []
    for model_name, model in models.items():
        metrics = train_evaluate_model(model, X_train, X_test, y_train, y_test)
        metrics["Model Name"] = model_name
        results.append(metrics)
    results_df = pd.DataFrame(results)
    return results_df

In [116]:
def detect_overfitting(metrics_df, threshold_ratio=1.5):
    overfitted_models = []
    
    for index, row in metrics_df.iterrows():
        train_rmse = row["Train RMSE"]
        test_rmse = row["Test RMSE"]
        model_name = row["Model Name"]
        
        # Verificar si train_rmse es cero para evitar división por cero
        if train_rmse == 0:
            # Si el RMSE de prueba es significativamente mayor que cero, considerar el modelo sobreentrenado
            if test_rmse > threshold_ratio:
                overfitted_models.append(model_name)
        else:
            # Evaluar si la métrica de prueba es significativamente mayor que la de entrenamiento
            if test_rmse / train_rmse > threshold_ratio:
                overfitted_models.append(model_name)
    
    return overfitted_models


In [117]:
# Seleccionar el mejor modelo basado en RMSE, MAE y R²
def select_best_model(metrics_df, overfitted_models):
    # Excluir los modelos detectados como sobreentrenados
    metrics_df = metrics_df[~metrics_df["Model Name"].isin(overfitted_models)]
    
    # Ordenar los modelos primero por RMSE de prueba, luego por MAE de prueba, y finalmente por R² de prueba
    sorted_metrics = metrics_df.sort_values(by=["Test RMSE", "Test MAE", "Test R2"], ascending=[True, True, False])
    best_model = sorted_metrics.iloc[0]  # Seleccionar la primera fila como el mejor modelo
    return best_model

In [151]:
# Pipeline principal para el entrenamiento y selección de modelos por cluster
def pipeline_for_clusters(df, target, models, test_size=0.2, vif_threshold=10,threshold_ratio=2):
    metrics_by_cluster = []
    best_models_by_cluster = []

    # Procesar por cada cluster
    for cluster in df['Cluster'].unique():
        print(f"Processing Cluster {cluster}")
        
        # Filtrar datos del cluster y extraer características temporales
        cluster_data = df[df['Cluster'] == cluster].copy()
        cluster_data = extract_time_features(cluster_data)
        
        # Eliminar la columna 'Cluster' del conjunto de datos
        cluster_data = cluster_data.drop(columns=['Cluster'])
        
        # Dividir en entrenamiento y prueba
        X_train, X_test, y_train, y_test = time_series_train_test_split(cluster_data, target, test_size)
        
        # Remover características con alto VIF
        #X_train, vif_data = remove_high_vif_features(X_train, threshold=vif_threshold)
        #X_test = X_test[X_train.columns]  # Asegurarse de que X_test tenga las mismas columnas que X_train
        
        # Evaluar todos los modelos para el cluster
        results_df = evaluate_models_for_cluster(X_train, X_test, y_train, y_test, models)
        
        # Detectar modelos sobreentrenados
        overfitted_models = detect_overfitting(results_df,threshold_ratio)
        
        # Seleccionar el mejor modelo del cluster
        best_model = select_best_model(results_df, overfitted_models)
        best_models_by_cluster.append(best_model)
        
        # Agregar resultados de todos los modelos al resumen general
        results_df["Cluster"] = cluster
        metrics_by_cluster.append(results_df)
        
        # Mostrar el mejor modelo para el cluster
        print(f"Best model for Cluster {cluster}:\n{best_model}\n")
    
    # Combinar todas las métricas en un solo DataFrame
    metrics_df = pd.concat(metrics_by_cluster, ignore_index=True)
    metrics_df.drop(columns=["Model"],inplace=True)
    best_models_df = pd.DataFrame(best_models_by_cluster)
    return metrics_df, best_models_df


REGRESION

In [155]:
# Cargar los datos y ejecutar el pipeline
average_time_series_by_cluster = pd.read_csv(data_dir("interim","average_time_series_by_cluster.csv"), parse_dates=["Date/Time"])
average_time_series_by_cluster.set_index('Date/Time', inplace=True)

# Definir la variable objetivo
target = 'Electricity:Facility [kW](Hourly)'

In [157]:
average_time_series_by_cluster[average_time_series_by_cluster.Cluster==0]

,Cluster,Electricity:Facility [kW](Hourly),Fans:Electricity [kW](Hourly),Cooling:Electricity [kW](Hourly),Heating:Electricity [kW](Hourly),InteriorLights:Electricity [kW](Hourly),InteriorEquipment:Electricity [kW](Hourly)
Date/Time,,,,,,,
2004-01-01 01:00:00,0,47.210969,5.789775,3.010854,4.637630,6.918895,10.669373
2004-01-01 02:00:00,0,47.047291,5.946731,3.048947,4.450657,6.588343,10.525933
2004-01-01 03:00:00,0,45.619393,5.862989,2.883068,5.406697,5.084587,10.308935
2004-01-01 04:00:00,0,45.697195,6.186896,2.968095,4.801340,5.084587,10.297614
2004-01-01 05:00:00,0,47.030928,5.832209,2.942949,5.603789,5.182567,10.482863
...,...,...,...,...,...,...,...
2004-12-31 19:00:00,0,78.902473,9.451682,6.007865,2.309477,14.749611,21.536755
2004-12-31 20:00:00,0,76.136075,8.184983,6.870403,2.518262,16.006585,19.579965
2004-12-31 21:00:00,0,77.329122,7.934092,7.675048,2.648414,16.504439,20.653062


In [ ]:
# Cargar modelos con configuraciones ajustadas
models = {
    "Linear Regression": LinearRegression(fit_intercept=True, n_jobs=-1),
    "Tree": DecisionTreeRegressor(max_depth=5, min_samples_split=5, random_state=42),
    "SVM": SVR(kernel='rbf', C=1.0, epsilon=0.1),
    "Random Forest": RandomForestRegressor(n_estimators=100, max_depth=5, min_samples_split=10, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, subsample=0.8, colsample_bytree=0.8, random_state=42)
}


In [152]:
# Ejecutar el pipeline
metrics_df, best_models_df = pipeline_for_clusters(average_time_series_by_cluster, target, models,threshold_ratio=2)

Processing Cluster 0
Best model for Cluster 0:
Model         XGBRegressor(base_score=None, booster=None, ca...
Train RMSE                                             1.677753
Train MAE                                              1.242559
Train R2                                               0.996641
Test RMSE                                              2.418129
Test MAE                                               1.875946
Test R2                                                0.990515
Model Name                                              XGBoost
Name: 4, dtype: object

Processing Cluster 1
Best model for Cluster 1:
Model         LinearRegression(n_jobs=-1)
Train RMSE                      34.509758
Train MAE                        27.98159
Train R2                         0.994399
Test RMSE                       39.835457
Test MAE                        31.613485
Test R2                          0.988051
Model Name              Linear Regression
Name: 0, dtype: object

Processing

In [154]:
best_models_df

,Model,Train RMSE,Train MAE,Train R2,Test RMSE,Test MAE,Test R2,Model Name
4,"XGBRegressor(base_score=None, booster=None, ca...",1.677753,1.242559,0.996641,2.418129,1.875946,0.990515,XGBoost
0,LinearRegression(n_jobs=-1),34.509758,27.981590,0.994399,39.835457,31.613485,0.988051,Linear Regression
4,"XGBRegressor(base_score=None, booster=None, ca...",13.238075,9.829072,0.996658,24.246907,19.310649,0.990215,XGBoost
0,LinearRegression(n_jobs=-1),6.566537,5.603544,0.999608,6.617337,5.642924,0.998601,Linear Regression


In [143]:
metrics_df

,Train RMSE,Train MAE,Train R2,Test RMSE,Test MAE,Test R2,Model Name,Cluster
0,3.501881,2.630465,0.985367,4.729990,4.131099,0.963707,Linear Regression,0
1,3.841605,2.943106,0.982390,4.219035,3.148905,0.971125,Tree,0
2,2.903144,2.111169,0.989943,3.808849,3.078405,0.976467,SVM,0
3,3.278744,2.520145,0.987172,3.820000,2.907997,0.976329,Random Forest,0
4,1.926543,1.447209,0.995571,2.956925,2.358443,0.985817,XGBoost,0
5,44.815280,35.383345,0.990555,57.250277,45.152314,0.975320,Linear Regression,1
6,42.676105,32.298716,0.991435,48.461779,36.986391,0.982316,Tree,1
7,53.928539,41.065141,0.986323,54.854075,42.001903,0.977343,SVM,1
8,39.659900,30.143872,0.992603,45.395798,35.049429,0.984483,Random Forest,1
9,20.881287,16.062453,0.997949,37.234342,28.488476,0.989561,XGBoost,1


In [ ]:
# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Configurar GridSearchCV
grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42),
                           param_grid=param_grid,
                           cv=3,  # Validación cruzada de 3 pliegues
                           n_jobs=-1,  # Usar todos los núcleos disponibles
                           scoring='neg_root_mean_squared_error',  # Usar RMSE
                           verbose=2)

# Realizar la búsqueda en cuadrícula
grid_search.fit(X_train_scaled_corr, y_train_corr)

# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_

In [ ]:
print("Mejores hiperparámetros:", best_params)

In [ ]:
# Entrenar el modelo con los mejores hiperparámetros
optimized_rf_model = RandomForestRegressor(**best_params, random_state=42)
optimized_rf_model.fit(X_train_scaled_corr, y_train_corr)

# Evaluar el modelo optimizado
rmse_train_opt = np.sqrt(mean_squared_error(y_train_corr, optimized_rf_model.predict(X_train_scaled_corr)))
r2_train_opt = r2_score(y_train_corr, optimized_rf_model.predict(X_train_scaled_corr))

rmse_test_opt = np.sqrt(mean_squared_error(y_test_corr, optimized_rf_model.predict(X_test_scaled_corr)))
r2_test_opt = r2_score(y_test_corr, optimized_rf_model.predict(X_test_scaled_corr))

# Resultados
optimized_rf_results = {
    "Optimized Random Forest": {
        'RMSE Train': rmse_train_opt,
        'R2 Train': r2_train_opt,
        'RMSE Test': rmse_test_opt,
        'R2 Test': r2_test_opt
    }
}

In [ ]:
# Mostrar los resultados
print("Resultados del modelo Random Forest optimizado:")
optimized_rf_results_df=pd.DataFrame(optimized_rf_results)
optimized_rf_results_df